In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F

In [2]:
def show(df, limit = 5):
    return df.toPandas().head(limit)

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [10]:
events = spark.read.csv(r"C:\Development\ultimateNakMuay\data\raw\wiki_events_onefc.csv", header=True)

In [13]:
events = events.withColumn(
    "date",
    F.when(
        F.to_date(F.col("date"), "MMMM d, yyyy").isNotNull(), F.to_date(F.col("date"), "MMMM d, yyyy")
        ).otherwise(F.to_date(F.col("date"), "d MMMM yyyy"))
        )\
    .withColumn("attendance", F.regexp_replace(F.col("attendance"), ",", ""))\
    .withColumn("attendance", F.col("attendance").cast(T.IntegerType()))\
    .withColumn("location", F.when(F.col("location") == "—", F.lit(None)).otherwise(F.col("location")))

In [16]:
events = events.withColumn("venue", F.when(F.col("venue") == "—", F.lit(None)).otherwise(F.col("venue")))

In [19]:
events = events.withColumn("location", F.split(F.col("location"), ","))\
    .withColumn("city", F.when(F.size(F.col("location")) == 3, F.element_at(F.col("location"), 1)))\
    .withColumn("state", F.when(
        F.size(F.col("location")) == 3, F.element_at(F.col("location"), 2)
        ).otherwise(F.element_at(F.col("location"), 1))
        )\
    .withColumn("country", F.element_at(F.col("location"), -1))\
    .withColumn("country", F.regexp_replace(F.col("country"), "[^a-zA-Z0-9 ]", ""))\
    .drop("location")

In [20]:
show(events)

,#,Event,date,venue,attendance,city,state,country
0,253,ONE Fight Night 13: Allazov vs. Grigorian,2023-08-05,Lumpinee Boxing Stadium,NaN,None,Bangkok,Thailand
1,252,ONE Fight Night 12,2023-07-15,Lumpinee Boxing Stadium,NaN,None,Bangkok,Thailand
2,251,ONE Friday Fights 22: Bhullar vs. Malykhin,2023-06-23,Lumpinee Boxing Stadium,NaN,None,Bangkok,Thailand
3,250,ONE Friday Fights 21,2023-06-16,Lumpinee Boxing Stadium,NaN,None,Bangkok,Thailand
4,249,ONE Fight Night 11: Eersel vs. Menshikov,2023-06-10,Lumpinee Boxing Stadium,NaN,None,Bangkok,Thailand


In [21]:
events = events.withColumnRenamed("#", "event_num")
for _ in events.columns:
    events = events.withColumnRenamed(_, _.lower())

In [23]:
cols = ["event_num", "event", "date", "venue", "city", "state", "country", "attendance"]

events = events.select(*cols)

In [24]:
show(events)

,event_num,event,date,venue,city,state,country,attendance
0,253,ONE Fight Night 13: Allazov vs. Grigorian,2023-08-05,Lumpinee Boxing Stadium,None,Bangkok,Thailand,NaN
1,252,ONE Fight Night 12,2023-07-15,Lumpinee Boxing Stadium,None,Bangkok,Thailand,NaN
2,251,ONE Friday Fights 22: Bhullar vs. Malykhin,2023-06-23,Lumpinee Boxing Stadium,None,Bangkok,Thailand,NaN
3,250,ONE Friday Fights 21,2023-06-16,Lumpinee Boxing Stadium,None,Bangkok,Thailand,NaN
4,249,ONE Fight Night 11: Eersel vs. Menshikov,2023-06-10,Lumpinee Boxing Stadium,None,Bangkok,Thailand,NaN
